In [ ]:
import numpy as np
import pandas as pd
import pathlib
import matplotlib.pyplot as plt

import seaborn as sns
sns.set_theme()
sns.set_context("talk", font_scale=1.0, rc={"font.family": "serif", "font.serif": ["Computer Modern Serif"]})

fig_size = (14,8)

In [ ]:
pytorch_networks = pathlib.Path("../model_outputs/")

### Get basic compare

In [ ]:
list_of_datasets = ["all_files"]
list_of_losses = ["rmse"]
pytorch_architectures_names = [    
    "Unet",
    "Unet++",
    "Deeplabv3",
    "Deeplabv3+",
    "FPNet",
    "PSPNet",
    "MANet",
    "LinkNet",
    "Pan",
    ]
pytorch_architectures = [    
    "unet",
    "unetplusplus",
    "deeplabv3",
    "deeplabv3plus",
    "fpn",
    "pspnet",
    "manet",
    "linknet",
    "pan",
    ]
pytorch_encoders = ["resnet18"]

In [ ]:
pytorch_df = pd.DataFrame([],columns=['Network', 'Architecture', 'Encoder', 'Dataset', 'IoU', 'Precision', 'Recall', 'Accuracy'])
for pytorch_architecture, pytorch_architecture_name in zip(pytorch_architectures, pytorch_architectures_names):
    for encoder in pytorch_encoders:
        for dataset in list_of_datasets:
            for loss in list_of_losses:
                pytorch_result = pd.read_csv(pytorch_networks / (pytorch_architecture+"_"+encoder+"_"+loss+"_"+dataset) / "evaluations" / "stats" / "general_stats.csv",index_col=0)
                pytorch_result["Network"] = "Pytorch_networks"
                pytorch_result["Architecture"] = pytorch_architecture_name
                pytorch_result["Encoder"] = "resnet18"
                pytorch_result["Dataset"] = dataset
                pytorch_df = pd.concat([pytorch_df,pytorch_result], axis=0)
pytorch_df = pytorch_df[["Network","Architecture","Encoder","Dataset","Accuracy"]]
pytorch_df

In [ ]:
valeo_row = pd.DataFrame([("Keras-Valeo","TiledSoilingNet","Resnet10","Valeo internal", 0.8735)],columns=pytorch_df.columns)
valeo_row

In [ ]:
complete_df = pd.concat([pytorch_df,valeo_row])
complete_df["Dataset"].replace("all_files","Woodscape", inplace=True)
complete_df = complete_df[["Architecture","Encoder","Dataset","Accuracy"]]
complete_df["Encoder"].replace("resnet18","Resnet18", inplace=True)
complete_df["Architecture"] = complete_df["Architecture"].str[0].str.upper() + complete_df["Architecture"].str[1:]
complete_df.sort_values(by="Accuracy", inplace=True, ascending=False)
complete_df

### Encoder compare

In [ ]:
list_of_datasets = ["all_files"]
list_of_losses = ["torch_cross_entropy"]
pytorch_architectures = [    
    "fpn"
    ]
pytorch_encoders = ["resnet18", "resnet34", "resnet50", "resnet101", "resnet152"]

In [ ]:
pytorch_df = pd.DataFrame([],columns=['Network', 'Architecture', 'Encoder', 'Dataset','Loss', 'IoU', 'Precision', 'Recall', 'Accuracy'])
for pytorch_architecture in pytorch_architectures:
    for encoder in pytorch_encoders:
        for dataset in list_of_datasets:
            for loss in list_of_losses:
                pytorch_result = pd.read_csv(pytorch_networks / (pytorch_architecture+"_"+encoder+"_"+loss+"_"+dataset) / "evaluations" / "stats" / "general_stats.csv",index_col=0)
                pytorch_result["Network"] = "Pytorch_networks"
                pytorch_result["Architecture"] = pytorch_architecture
                pytorch_result["Encoder"] = encoder
                pytorch_result["Dataset"] = dataset
                pytorch_result["Loss"] = loss
                pytorch_df = pd.concat([pytorch_df,pytorch_result], axis=0)
#pytorch_df = pytorch_df[["Network","Architecture","Encoder","Dataset","Accuracy"]]
pytorch_df

### Loss correlations

In [ ]:
list_of_datasets = ["all_files"]
list_of_losses = ["torch_cross_entropy","dice_loss","focal_loss","rmse"]
pytorch_architectures = [    
    "fpn"
    ]
pytorch_encoders = ["resnet18"]

In [ ]:
pytorch_df = pd.DataFrame([],columns=['Architecture', 'Encoder', 'Loss', 'Dataset', 'IoU', 'Precision', 'Recall', 'Accuracy'])
for pytorch_architecture in pytorch_architectures:
    for encoder in pytorch_encoders:
        for dataset in list_of_datasets:
            for loss in list_of_losses:
                pytorch_result = pd.read_csv(pytorch_networks / (pytorch_architecture+"_"+encoder+"_"+loss+"_"+dataset) / "evaluations" / "stats" / "general_stats.csv",index_col=0)
                pytorch_result["Architecture"] = pytorch_architecture
                pytorch_result["Encoder"] = encoder
                pytorch_result["Dataset"] = dataset
                pytorch_result["Loss"] = loss
                pytorch_df = pd.concat([pytorch_df,pytorch_result], axis=0)
pytorch_df = pytorch_df[["Architecture","Encoder","Loss","Dataset","Accuracy"]]
pytorch_df

### Dataset correlations

In [ ]:
list_of_datasets = ["all_files", "correct_files", "correct_clear_files", "correct_clear_strict_files"]
dataset_sorting = {"all_files":0, "correct_files":1, "correct_clear_files":2, "correct_clear_strict_files":3}
list_of_losses = ["torch_cross_entropy"]
pytorch_architectures = [    
    "unet",
    "fpn",
    "pspnet",
    "manet",
    "linknet",
    "pan",
    ]
pytorch_architectures_names = [    
    "Unet",
    "FPNet",
    "PSPNet",
    "MANet",
    "LinkNet",
    "Pan",
    ]
pytorch_encoders = ["resnet18"]

In [ ]:
pytorch_df = pd.DataFrame([],columns=['Architecture', 'Encoder', 'Loss', 'Dataset', 'IoU', 'Precision', 'Recall', 'Accuracy'])
for pytorch_architecture, pytorch_architecture_name in zip(pytorch_architectures, pytorch_architectures_names):
    for encoder in pytorch_encoders:
        for dataset in list_of_datasets:
            for loss in list_of_losses:
                pytorch_result = pd.read_csv(pytorch_networks / (pytorch_architecture+"_"+encoder+"_"+loss+"_"+dataset) / "evaluations" / "stats" / "general_stats.csv",index_col=0)
                pytorch_result["Architecture"] = pytorch_architecture_name
                pytorch_result["Encoder"] = encoder
                pytorch_result["Dataset"] = dataset
                pytorch_result["Loss"] = loss
                pytorch_df = pd.concat([pytorch_df,pytorch_result], axis=0)
pytorch_df = pytorch_df[["Architecture","Encoder","Loss","Dataset","Accuracy"]]
pytorch_df.sort_values(by="Dataset", ascending=True, inplace=True, key=lambda x: x.map(dataset_sorting))
pytorch_df

In [ ]:
plt.figure(figsize = (14,8))
plt.title("Test accuracy")
for pytorch_architecture in pytorch_architectures_names:
    combination_df = pytorch_df[pytorch_df["Architecture"]==pytorch_architecture]
    plt.plot(combination_df["Dataset"], combination_df["Accuracy"], label = f"{pytorch_architecture}")
#plt.ylim([0,1])
plt.xlabel("Training set")
plt.ylabel("Accuracy")
plt.legend()
plt.savefig("test_accuracy_2.pdf")
plt.show()

### Training times

In [ ]:
pytorch_architectures = [    
    "unet",
    "fpn",
    "pspnet",
    "manet",
    "linknet",
    "pan",
    ]
pytorch_architectures_names = [    
    "Unet",
    "FPNet",
    "PSPNet",
    "MANet",
    "LinkNet",
    "Pan",
    ]

In [ ]:
training_df = pd.read_csv("../model_outputs/wandb_export_2.csv")
training_df = training_df[(training_df["encoder"] =="resnet18") | (training_df["encoder"].isna())]
training_df = training_df[(training_df["framework"] =="pytorch_networks")]
training_df = training_df[(training_df["loss"] =="torch_cross_entropy")]
training_df.drop_duplicates(subset=["model","encoder","dataset"], inplace=True,keep="first")
training_df.sort_values(by=["model","encoder","dataset"],ascending=True, inplace=True)
training_df

In [ ]:
training_df = training_df[(training_df["model"].isin(pytorch_architectures))]
training_df["Runtime_relative"] = training_df["Runtime"]/training_df["epochs"]
training_df

In [ ]:
plt.figure(figsize = (14,8))
for pytorch_architecture,pytorch_architecture_name in zip(pytorch_architectures, pytorch_architectures_names):
    combination_df = training_df[training_df["model"]==pytorch_architecture]
    training_times = []
    for dataset_name in list_of_datasets:
        training_times.append(combination_df[combination_df["dataset"]==dataset_name]["Runtime_relative"].item())
    plt.plot(list_of_datasets, training_times, label = f"{pytorch_architecture_name}")
#plt.ylim([0,1])
plt.xlabel("Training set")
plt.ylabel("Training times in seconds")
plt.legend()
plt.title("Training time per epoch")
plt.savefig("training_times_2.pdf")
plt.show()